In [ ]:
import os
#os.environ["TF_CPP_MIN_LOG_LEVEL"] = "1"
import matplotlib.pyplot as plt
import cv2
import numpy as np
import tensorflow as tf
from tensorflow import keras
from keras import layers
import einops
import random
from tensorflow.keras.utils import to_categorical

In [ ]:
gpus = tf.config.list_physical_devices('GPU')
if gpus:
  try:
    # Currently, memory growth needs to be the same across GPUs
    for gpu in gpus:
      tf.config.experimental.set_memory_growth(gpu, True)
    logical_gpus = tf.config.list_logical_devices('GPU')
    print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPUs")
  except RuntimeError as e:
    # Memory growth must be set before GPUs have been initialized
    print(e)

In [ ]:
ucf_url = "./crime_data/UCF_Crimes-Train-Test-Split/Action_Regnition_splits/"
video_url = "./crime_data/Anomaly-Videos-Part-1/"

In [ ]:
classes_file = open(f'{ucf_url}ClassIDs.txt',"r")
classes_ids = classes_file.read().split("\n")
classes = []
class_ids = []
class_dict = {}
for elem in classes_ids:
    x = elem.strip().split(' ')
    if len(x) == 2:
        classes.append(x[0])
        class_ids.append(x[1])
        class_dict[x[0]] = int(x[1])-1
print(classes)
print(class_ids)
print(class_dict)

In [ ]:
len(classes)

In [ ]:
files_main = open(f'{ucf_url}train_001.txt','r').read().split(' \n')
len(files_main)

In [ ]:
def format_frames(frame, output_size):
    frame = tf.image.convert_image_dtype(frame, tf.float32)
    frame = tf.image.resize_with_pad(frame, *output_size)
    frane = tf.image.central_crop(frame,0.9)
    frame = tf.image.random_crop(value=frame, size=(200,200,3))
    return frame

def frames_from_video_file(video_path, n_frames, output_size = (240,320)):
  video_path = f'{video_url}{video_path}'
  result = []
  src = cv2.VideoCapture(str(video_path))  
  start = 0
  src.set(cv2.CAP_PROP_POS_FRAMES, start)
  frame_step = int((src.get(cv2.CAP_PROP_FRAME_COUNT) // n_frames) - 1)

  ret, frame = src.read()
  result.append(format_frames(frame, output_size))

  for _ in range(n_frames - 1):
    for _ in range(frame_step):
      ret, frame = src.read()
    if ret:
      frame = format_frames(frame, output_size)
      result.append(frame)
    else:
      result.append(np.zeros_like(result[0]))
  src.release()
  result = np.array(result)[..., [2, 1, 0]]

  return result

class FrameGenerator:
  def __init__(self, paths, n_frames, class_dictionary, training = False):
    self.paths = paths
    self.n_frames = n_frames
    self.training = training
    self.class_ids_for_name = class_dictionary

  def get_files_and_class_names(self):
    classes = [i.split('/')[0] for i in self.paths] 
    return self.paths, classes

  def __call__(self):
    video_paths, classes = self.get_files_and_class_names()

    pairs = list(zip(video_paths, classes))

    if self.training:
      random.shuffle(pairs)

    for path, name in pairs:
      video_frames = frames_from_video_file(path, self.n_frames)
      label = self.class_ids_for_name[name]
      yield video_frames, label

In [ ]:
random.shuffle(files_main)
train_set = files_main
val_set = [train_set.pop(random.randint(0,len(train_set)-1)) for _ in range(int(len(files_main)*0.20))]
print(len(val_set))
print(len(train_set))

In [ ]:
fg = FrameGenerator(train_set, 30,class_dict, training=True)

frames, label = next(fg())

print(f"Shape: {frames.shape}")
print(f"Label: {label}")

In [ ]:
output_signature = (tf.TensorSpec(shape = (30, 200, 200, 3), dtype = tf.float32),
                    tf.TensorSpec(shape = (), dtype = tf.int16))
train_ds = tf.data.Dataset.from_generator(FrameGenerator(train_set, 30,class_dict, training=True),
                                          output_signature = output_signature)

val_ds = tf.data.Dataset.from_generator(FrameGenerator(val_set, 30, class_dict),
                                        output_signature = output_signature)
AUTOTUNE = tf.data.AUTOTUNE

train_ds = train_ds.cache().shuffle(500).repeat().prefetch(buffer_size = AUTOTUNE)
val_ds = val_ds.cache().shuffle(200).repeat().prefetch(buffer_size = AUTOTUNE)
batchsize = 1
train_ds = train_ds.batch(batch_size=batchsize)
val_ds = val_ds.batch(batch_size=batchsize)

In [ ]:
from tensorflow import keras
from keras import layers
from keras import regularizers
from keras.layers import SpatialDropout3D

In [ ]:
# input_shape = (None,30,200,200,3)
# model = keras.Sequential([
#     layers.Rescaling(scale=1./255, input_shape=(30,200,200,3)),
#     layers.Conv3D(16,(3,7,7), activation="relu"),
#     layers.Conv3D(32,(3,3,3), activation="relu"),
#     layers.Conv3D(64,(3,3,3), activation="relu"),
#     layers.GlobalAveragePooling3D(),
#     layers.Flatten(),
#     layers.Dense(14)
# ])

net = keras.applications.EfficientNetB0(include_top=False)
net.trainable = False
model = keras.Sequential([
    layers.Rescaling(scale=1./255),
    layers.TimeDistributed(net),
    layers.Dense(14),
    layers.GlobalAveragePooling3D()
])

model.summary()

In [ ]:
model.compile(loss = keras.losses.SparseCategoricalCrossentropy(from_logits=True), 
              optimizer = keras.optimizers.Adam(0.1), 
              metrics = ['accuracy'])

In [ ]:
from keras.callbacks import ReduceLROnPlateau, EarlyStopping
reduce_lr = ReduceLROnPlateau(monitor='val_loss',patience=5)
earlystopping = EarlyStopping(patience=2,monitor="val_accuracy", restore_best_weights=True)

In [ ]:
# history = model.fit(x = train_ds,
#                     epochs = 20, 
#                     validation_data = val_ds,
#                     steps_per_epoch=426 //batchsize,
#                     validation_steps=106 //batchsize,
#                     callbacks=[reduce_lr,earlystopping])
history = model.fit(x = train_ds,
                    epochs = 10, 
                    validation_data = val_ds,
                    callbacks=[earlystopping])